# Project GDP
## GitHub Repo: https://github.com/JackBeerman/GDP

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import zipfile
import io
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import psycopg2
from sqlalchemy import create_engine

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State

import plotly.figure_factory as ff
import plotly.express as px
import openai
import asyncio
from openai import AsyncOpenAI
import plotly.graph_objs as go

key = os.getenv("OPENAI_API_KEY")
openai.key = key

## Data

* The World Bank https://data.worldbank.org/
* World Integrated Trade Solution https://wits.worldbank.org/
* World Trade Organization https://www.wto.org/english/res_e/statis_e/statis_e.htm
* Our World in Data https://ourworldindata.org/trade-and-globalization
* International Monetary Fund https://data.imf.org/?sk=9d6028d4-f14a-464c-a2f2-59b2cd424b85
* Human Development Reports https://hdr.undp.org/

## Preprocess World Bank

In [2]:
noncountries = ["Arab World", "Central Europe and the Baltics",
"Caribbean small states",
"East Asia & Pacific (excluding high income)",
"Early-demographic dividend","East Asia & Pacific",
"Europe & Central Asia (excluding high income)",
"Europe & Central Asia", "Euro area",
"European Union","Fragile and conflict affected situations",
"High income",
"Heavily indebted poor countries (HIPC)","IBRD only",
"IDA & IBRD total",
"IDA total","IDA blend","IDA only",
"Latin America & Caribbean (excluding high income)",
"Latin America & Caribbean",
"Least developed countries: UN classification",
"Low income","Lower middle income","Low & middle income",
"Late-demographic dividend","Middle East & North Africa",
"Middle income",
"Middle East & North Africa (excluding high income)",
"North America","OECD members",
"Other small states","Pre-demographic dividend",
"Pacific island small states",
"Post-demographic dividend",
"Sub-Saharan Africa (excluding high income)",
"Sub-Saharan Africa",
"Small states","East Asia & Pacific (IDA & IBRD)",
"Europe & Central Asia (IDA & IBRD)",
"Latin America & Caribbean (IDA & IBRD)",
"Middle East & North Africa (IDA & IBRD)","South Asia",
"South Asia (IDA & IBRD)",
"Sub-Saharan Africa (IDA & IBRD)",
"Upper middle income", "World"]

In [3]:
exports = pd.read_csv("exports/goods_exported.csv", skiprows=4)
exports.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"Goods exports (BoP, current US$)",BX.GSR.MRCH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.545405e+08,3.296600e+08,2.840210e+08,1.346004e+08,1.848016e+08,1.307081e+08,8.733158e+07,1.230030e+08,2.052746e+08,NaN
1,Africa Eastern and Southern,AFE,"Goods exports (BoP, current US$)",BX.GSR.MRCH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.328337e+11,1.807787e+11,1.713935e+11,1.954152e+11,2.161620e+11,2.026909e+11,1.805053e+11,2.585645e+11,2.892419e+11,NaN
2,Afghanistan,AFG,"Goods exports (BoP, current US$)",BX.GSR.MRCH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,6.417674e+08,5.778598e+08,6.142176e+08,7.839629e+08,8.752427e+08,8.638344e+08,7.767345e+08,NaN,NaN,NaN
3,Africa Western and Central,AFW,"Goods exports (BoP, current US$)",BX.GSR.MRCH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.657213e+11,1.084194e+11,9.644857e+10,1.192904e+11,1.424634e+11,1.496790e+11,1.182803e+11,1.434662e+11,NaN,NaN
4,Angola,AGO,"Goods exports (BoP, current US$)",BX.GSR.MRCH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.916988e+10,3.318113e+10,2.758888e+10,3.461345e+10,4.075777e+10,3.472556e+10,2.093744e+10,3.358149e+10,5.003799e+10,NaN


In [4]:
#print(exports.columns)

In [5]:
exports = exports.drop(columns=['Indicator Code', 'Unnamed: 67'])
#print(exports.columns)

In [6]:
exports = exports.rename(columns={"Country Code": "country_code", "Country Name": "country_name_exp", "Indicator Name": 'feature'})
#exports

In [7]:
exports = exports.query('country_name_exp != @noncountries')
#exports

In [8]:
imports = pd.read_csv("imports/import_goods_services.csv", skiprows=4)
#imports.head()

In [9]:
#print(imports.columns)

In [10]:
imports = imports.drop(columns=['Indicator Code', 'Unnamed: 67'])
#print(imports.columns)

In [11]:
imports = imports.rename(columns={"Country Code": "country_code", "Country Name": "country_name_imp", "Indicator Name": 'feature'})
#imports

In [12]:
imports = imports.query('country_name_imp != @noncountries')
#imports

#### GDP

In [13]:
gdp = pd.read_csv("gdp/GDP.csv", skiprows=4)
#gdp.head()

In [14]:
#print(gdp.columns)

In [15]:
gdp = gdp.drop(columns=['Indicator Code', 'Unnamed: 67'])
#print(gdp.columns)

In [16]:
gdp = gdp.rename(columns={"Country Code": "country_code", "Country Name": "country_name_gdp", "Indicator Name": 'feature'})
#gdp

In [17]:
gdp = gdp.query('country_name_gdp != @noncountries')
#gdp

##### Tariffs

In [18]:
tariffs = pd.read_csv("tariffs/tarrifs.csv", skiprows=4)
#tariffs.head()

In [19]:
#print(tariffs.columns)

In [20]:
tariffs = tariffs.drop(columns=['Indicator Code', 'Unnamed: 67'])
#print(tariffs.columns)

In [21]:
tariffs = tariffs.rename(columns={"Country Code": "country_code", "Country Name": "country_name_tar", "Indicator Name": 'feature'})
#tariffs

In [22]:
tariffs = tariffs.query('country_name_tar != @noncountries')
#tariffs

# Melt and conform to one data frame

In [23]:
exports = pd.melt(exports, id_vars=['country_name_exp', 'country_code'
                         ,'feature'], var_name='year', value_name='value')
#exports

In [24]:
exports = exports.pivot_table(index=['country_name_exp','country_code', 'year'], columns='feature', values='value').reset_index()
 
 
exports.columns.name = None
 
 
exports = exports.reset_index(drop=True)

In [25]:
exports['year'] = pd.to_numeric(exports['year'])

In [26]:
imports = pd.melt(imports, id_vars=['country_name_imp', 'country_code'
                         ,'feature'], var_name='year', value_name='value')
#imports

In [27]:
imports = imports.pivot_table(index=['country_name_imp','country_code', 'year'], columns='feature', values='value').reset_index()
 
 
imports.columns.name = None
 
 
imports = imports.reset_index(drop=True)

In [28]:
imports['year'] = pd.to_numeric(imports['year'])

# Merge 1

In [29]:
merge1 = pd.merge(exports, imports, on = ['country_code', 'year'],
                  how = 'inner')

In [30]:
gdp = pd.melt(gdp, id_vars=['country_name_gdp', 'country_code'
                         ,'feature'], var_name='year', value_name='value')
#gdp

In [31]:
gdp = gdp.pivot_table(index=['country_name_gdp','country_code', 'year'], columns='feature', values='value').reset_index()
 
 
gdp.columns.name = None
 
 
gdp = gdp.reset_index(drop=True)

In [32]:
gdp['year'] = pd.to_numeric(gdp['year'])

In [33]:
tariffs = pd.melt(tariffs, id_vars=['country_name_tar', 'country_code'
                         ,'feature'], var_name='year', value_name='value')
#tariffs

In [34]:
tariffs = tariffs.pivot_table(index=['country_name_tar','country_code', 'year'], columns='feature', values='value').reset_index()
 
 
tariffs.columns.name = None
 
 
tariffs = tariffs.reset_index(drop=True)

In [35]:
tariffs['year'] = pd.to_numeric(tariffs['year'])

# Merge 2

In [36]:
merge2 = pd.merge(gdp, tariffs, on = ['country_code', 'year'],
                  how = 'inner')

# Merge 3

In [37]:
merge = pd.merge(merge1, merge2, on = ['country_code', 'year'],
                  how = 'inner')

In [38]:
merge.columns

Index(['country_name_exp', 'country_code', 'year',
       'Goods exports (BoP, current US$)', 'country_name_imp',
       'Imports of goods, services and primary income (BoP, current US$)',
       'country_name_gdp', 'GDP (current US$)', 'country_name_tar',
       'Tariff rate, most favored nation, weighted mean, all products (%)'],
      dtype='object')

In [39]:
merge = merge.drop(columns=['country_name_imp', 'country_name_gdp', 'country_name_tar'])
merge

,country_name_exp,country_code,year,"Goods exports (BoP, current US$)","Imports of goods, services and primary income (BoP, current US$)",GDP (current US$),"Tariff rate, most favored nation, weighted mean, all products (%)"
0,Afghanistan,AFG,2008,5.632992e+08,3.873397e+09,1.024977e+10,5.00
1,Afghanistan,AFG,2012,4.755310e+08,9.891159e+09,2.020357e+10,5.50
2,Afghanistan,AFG,2013,5.057926e+08,1.018433e+10,2.056449e+10,5.50
3,Afghanistan,AFG,2018,8.752427e+08,8.086339e+09,1.841886e+10,5.72
4,Albania,ALB,1997,9.090000e+07,6.976125e+08,2.258514e+09,14.41
...,...,...,...,...,...,...,...
3461,Zimbabwe,ZWE,2010,3.245066e+09,6.593559e+09,1.204166e+10,16.60
3462,Zimbabwe,ZWE,2011,4.527563e+09,9.528102e+09,1.410192e+10,13.49
3463,Zimbabwe,ZWE,2012,3.963773e+09,8.683377e+09,1.711485e+10,14.76
3464,Zimbabwe,ZWE,2015,3.577478e+09,7.584347e+09,1.996312e+10,12.86


In [40]:
merge.columns

Index(['country_name_exp', 'country_code', 'year',
       'Goods exports (BoP, current US$)',
       'Imports of goods, services and primary income (BoP, current US$)',
       'GDP (current US$)',
       'Tariff rate, most favored nation, weighted mean, all products (%)'],
      dtype='object')

In [41]:
merge

,country_name_exp,country_code,year,"Goods exports (BoP, current US$)","Imports of goods, services and primary income (BoP, current US$)",GDP (current US$),"Tariff rate, most favored nation, weighted mean, all products (%)"
0,Afghanistan,AFG,2008,5.632992e+08,3.873397e+09,1.024977e+10,5.00
1,Afghanistan,AFG,2012,4.755310e+08,9.891159e+09,2.020357e+10,5.50
2,Afghanistan,AFG,2013,5.057926e+08,1.018433e+10,2.056449e+10,5.50
3,Afghanistan,AFG,2018,8.752427e+08,8.086339e+09,1.841886e+10,5.72
4,Albania,ALB,1997,9.090000e+07,6.976125e+08,2.258514e+09,14.41
...,...,...,...,...,...,...,...
3461,Zimbabwe,ZWE,2010,3.245066e+09,6.593559e+09,1.204166e+10,16.60
3462,Zimbabwe,ZWE,2011,4.527563e+09,9.528102e+09,1.410192e+10,13.49
3463,Zimbabwe,ZWE,2012,3.963773e+09,8.683377e+09,1.711485e+10,14.76
3464,Zimbabwe,ZWE,2015,3.577478e+09,7.584347e+09,1.996312e+10,12.86


In [42]:
merge = merge.rename(columns={"Goods exports (BoP, current US$)": "exports", "Imports of goods, services and primary income (BoP, current US$)": "imports", "GDP (current US$)": "gdp", "Tariff rate, most favored nation, weighted mean, all products (%)": 'tariff_rate'})
merge

,country_name_exp,country_code,year,exports,imports,gdp,tariff_rate
0,Afghanistan,AFG,2008,5.632992e+08,3.873397e+09,1.024977e+10,5.00
1,Afghanistan,AFG,2012,4.755310e+08,9.891159e+09,2.020357e+10,5.50
2,Afghanistan,AFG,2013,5.057926e+08,1.018433e+10,2.056449e+10,5.50
3,Afghanistan,AFG,2018,8.752427e+08,8.086339e+09,1.841886e+10,5.72
4,Albania,ALB,1997,9.090000e+07,6.976125e+08,2.258514e+09,14.41
...,...,...,...,...,...,...,...
3461,Zimbabwe,ZWE,2010,3.245066e+09,6.593559e+09,1.204166e+10,16.60
3462,Zimbabwe,ZWE,2011,4.527563e+09,9.528102e+09,1.410192e+10,13.49
3463,Zimbabwe,ZWE,2012,3.963773e+09,8.683377e+09,1.711485e+10,14.76
3464,Zimbabwe,ZWE,2015,3.577478e+09,7.584347e+09,1.996312e+10,12.86


In [43]:
print(merge["country_name_exp"])

0       Afghanistan
1       Afghanistan
2       Afghanistan
3       Afghanistan
4           Albania
           ...     
3461       Zimbabwe
3462       Zimbabwe
3463       Zimbabwe
3464       Zimbabwe
3465       Zimbabwe
Name: country_name_exp, Length: 3466, dtype: object


## Preprocess Human Development Reports

need to add openpyxl to requirements

In [44]:
hdi = pd.read_excel("HDR21-22_Statistical_Annex_HDI_Table.xlsx")

In [45]:
hdi.head()

,HDI rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,Previous HDI rank
0,1,Switzerland,0.962,83.9872,16.500299,13.85966,66933.00454,5,3
1,2,Norway,0.961,83.2339,18.185200,13.00363,64660.10622,6,1
2,3,Iceland,0.959,82.6782,19.163059,13.76717,55782.04981,11,2
3,4,"Hong Kong, China (SAR)",0.952,85.4734,17.278170,12.22621,62606.84540,6,4
4,5,Australia,0.951,84.5265,21.054590,12.72682,49238.43335,18,5


In [46]:
hdi.dtypes

HDI rank                                    int64
Country                                    object
Human Development Index (HDI)             float64
Life expectancy at birth                  float64
Expected years of schooling               float64
Mean years of schooling                   float64
Gross national income (GNI) per capita    float64
GNI per capita rank minus HDI rank          int64
Previous HDI rank                           int64
dtype: object

In [47]:
hdi.columns = hdi.columns.str.strip()

In [48]:
hdi = hdi.rename(columns={"HDI rank": "rank","Country": "country_name_hdi", "Human Development Index (HDI)": "hdi_value", "Life expectancy at birth": 'life_exp', "Expected years of schooling": 'exp_years_of_school', "Mean years of schooling": 'mean_years_of_school', "Gross national income (GNI) per capita": 'gni', "GNI per capita rank minus HDI rank": 'gni_minus_hdi', "Previous HDI rank": 'old_hdi' })
hdi

,rank,country_name_hdi,hdi_value,life_exp,exp_years_of_school,mean_years_of_school,gni,gni_minus_hdi,old_hdi
0,1,Switzerland,0.962,83.9872,16.500299,13.859660,66933.004540,5,3
1,2,Norway,0.961,83.2339,18.185200,13.003630,64660.106220,6,1
2,3,Iceland,0.959,82.6782,19.163059,13.767170,55782.049810,11,2
3,4,"Hong Kong, China (SAR)",0.952,85.4734,17.278170,12.226210,62606.845400,6,4
4,5,Australia,0.951,84.5265,21.054590,12.726820,49238.433350,18,5
...,...,...,...,...,...,...,...,...,...
186,187,Burundi,0.426,61.6627,10.722722,3.129267,731.786709,4,187
187,188,Central African Republic,0.404,53.8947,8.040172,4.334000,966.058611,1,188
188,189,Niger,0.400,61.5763,6.957112,2.116717,1239.866936,-3,189
189,190,Chad,0.394,52.5254,8.035914,2.573774,1364.169417,-7,190


In [49]:
hdi.columns

Index(['rank', 'country_name_hdi', 'hdi_value', 'life_exp',
       'exp_years_of_school', 'mean_years_of_school', 'gni', 'gni_minus_hdi',
       'old_hdi'],
      dtype='object')

### latitude and longititude of country for 

### https://www.kaggle.com/datasets/arviinndn/countries

In [50]:
countries = pd.read_csv("countries.csv")
countries

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


In [51]:
hdi

,rank,country_name_hdi,hdi_value,life_exp,exp_years_of_school,mean_years_of_school,gni,gni_minus_hdi,old_hdi
0,1,Switzerland,0.962,83.9872,16.500299,13.859660,66933.004540,5,3
1,2,Norway,0.961,83.2339,18.185200,13.003630,64660.106220,6,1
2,3,Iceland,0.959,82.6782,19.163059,13.767170,55782.049810,11,2
3,4,"Hong Kong, China (SAR)",0.952,85.4734,17.278170,12.226210,62606.845400,6,4
4,5,Australia,0.951,84.5265,21.054590,12.726820,49238.433350,18,5
...,...,...,...,...,...,...,...,...,...
186,187,Burundi,0.426,61.6627,10.722722,3.129267,731.786709,4,187
187,188,Central African Republic,0.404,53.8947,8.040172,4.334000,966.058611,1,188
188,189,Niger,0.400,61.5763,6.957112,2.116717,1239.866936,-3,189
189,190,Chad,0.394,52.5254,8.035914,2.573774,1364.169417,-7,190


In [52]:
merge_full = pd.merge(countries, hdi, left_on='name', right_on='country_name_hdi', 
                      how='inner')
merge_full.columns

Index(['country', 'latitude', 'longitude', 'name', 'rank', 'country_name_hdi',
       'hdi_value', 'life_exp', 'exp_years_of_school', 'mean_years_of_school',
       'gni', 'gni_minus_hdi', 'old_hdi'],
      dtype='object')

In [53]:
merge_full
merge_full = merge_full.rename(columns={'Human Development Index (HDI)': 'hdi_val'})
merge_full = merge_full.drop('name', axis=1)

In [54]:
merge_full

,country,latitude,longitude,rank,country_name_hdi,hdi_value,life_exp,exp_years_of_school,mean_years_of_school,gni,gni_minus_hdi,old_hdi
0,AD,42.546245,1.601554,40,Andorra,0.858,80.3684,13.300239,10.555120,51166.626610,-19,45
1,AE,23.424076,53.847818,26,United Arab Emirates,0.911,78.7104,15.717690,12.694030,62573.591810,-15,25
2,AF,33.939110,67.709953,180,Afghanistan,0.478,61.9824,10.263844,2.985070,1824.190915,-2,177
3,AG,17.060816,-61.796428,71,Antigua and Barbuda,0.788,78.4968,14.184626,9.293741,16792.365950,2,71
4,AL,41.153332,20.168331,67,Albania,0.796,76.4626,14.448000,11.286455,14131.110390,17,68
...,...,...,...,...,...,...,...,...,...,...,...,...
162,WS,-13.759029,-172.104629,111,Samoa,0.707,72.7675,12.418859,11.403800,5307.953374,24,112
163,YE,15.552727,48.516388,183,Yemen,0.455,63.7534,9.098710,3.200000,1314.270189,1,183
164,ZA,-30.559482,22.937506,109,South Africa,0.713,62.3410,13.643710,11.373160,12948.373250,-17,102
165,ZM,-13.133897,27.849332,154,Zambia,0.565,61.2234,10.928760,7.187091,3217.767739,7,154


In [55]:
merge_full = merge_full[merge_full['country_name_hdi'].isin(merge['country_name_exp'])]
merge_full = merge_full.drop('gni_minus_hdi', axis=1)
merge_full

,country,latitude,longitude,rank,country_name_hdi,hdi_value,life_exp,exp_years_of_school,mean_years_of_school,gni,gni_minus_hdi,old_hdi
2,AF,33.939110,67.709953,180,Afghanistan,0.478,61.9824,10.263844,2.985070,1824.190915,-2,177
3,AG,17.060816,-61.796428,71,Antigua and Barbuda,0.788,78.4968,14.184626,9.293741,16792.365950,2,71
4,AL,41.153332,20.168331,67,Albania,0.796,76.4626,14.448000,11.286455,14131.110390,17,68
5,AM,40.069099,45.038189,85,Armenia,0.759,72.0431,13.116760,11.330300,13157.993900,4,87
6,AO,-11.202692,17.873887,148,Angola,0.586,61.6434,12.172100,5.417391,5465.617791,-14,149
...,...,...,...,...,...,...,...,...,...,...,...,...
161,VU,-15.376706,166.959158,140,Vanuatu,0.607,70.4490,11.535315,7.064846,3085.411633,23,142
162,WS,-13.759029,-172.104629,111,Samoa,0.707,72.7675,12.418859,11.403800,5307.953374,24,112
164,ZA,-30.559482,22.937506,109,South Africa,0.713,62.3410,13.643710,11.373160,12948.373250,-17,102
165,ZM,-13.133897,27.849332,154,Zambia,0.565,61.2234,10.928760,7.187091,3217.767739,7,154


In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


features = ['life_exp', 'exp_years_of_school', 'mean_years_of_school', 'gni']
target = 'rank'

# Splitting the data
X = merge_full[features]
y = merge_full[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
model = LinearRegression()
model.fit(X_train, y_train)

# Making Predictions
y_pred = model.predict(X_test)

# Model Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 49.20360884419671
R-squared: 0.9832638214092569


In [57]:
import pickle

# Save the trained model to a file
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Database Time

In [58]:
dbserver = psycopg2.connect(
    host = 'postgres',
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    port = 5432
)
dbserver.autocommit = True

In [59]:
cursor = dbserver.cursor()

In [61]:
try:
    cursor.execute('CREATE DATABASE world')
except:
    cursor.execute('DROP DATABASE world')
    cursor.execute('CREATE DATABASE world')

In [62]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'world'
))

In [63]:
### review if  I even need to merge every 
merge.to_sql('year_country', con=engine, index=False, chunksize=1000, if_exists = 'replace')
merge_full.to_sql('hdi', con=engine, index=False, chunksize=1000, if_exists = 'replace')

144

In [64]:
myquery = f'''
    SELECT country_code
    FROM year_country
    '''

mydf = pd.read_sql_query(myquery, con=engine)

In [65]:
mydf

,country_code
0,AFG
1,AFG
2,AFG
3,AFG
4,ALB
...,...
3461,ZWE
3462,ZWE
3463,ZWE
3464,ZWE


# DBDOCS

In [66]:
#for col in worlddf.columns:
#    print(col, worlddf[col].dtype)

## gives info for DBdocs

## two tables one with key country and year
## the other with just country 

In [67]:
for col in merge.columns:
    print(col, merge[col].dtype)

country_name_exp object
country_code object
year int64
exports float64
imports float64
gdp float64
tariff_rate float64


In [68]:
for col in merge_full.columns:
    print(col, merge_full[col].dtype)

country object
latitude float64
longitude float64
rank int64
country_name_hdi object
hdi_value float64
life_exp float64
exp_years_of_school float64
mean_years_of_school float64
gni float64
gni_minus_hdi int64
old_hdi int64


# DBDOCS

https://dbdocs.io/jtb3sud/WorldTrade?view=relationships